input: diffdock_before.txt, receptor_clean.pdb

output: Lipinski_before.txt

In [ ]:
#convert smiles string to sdf
import os
from rdkit import Chem
from rdkit.Chem import AllChem

# Create output directory
output_dir = "sdf_output"
os.makedirs(output_dir, exist_ok=True)

# Read SMILES from file
with open("diffdock_before.txt", "r") as file:
    smiles_list = [line.strip() for line in file if line.strip()]

# Generate SDF files
for idx, smiles in enumerate(smiles_list):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Skipping invalid SMILES at line {idx + 1}: {smiles}")
        continue
    mol = Chem.AddHs(mol)
    if AllChem.EmbedMolecule(mol, AllChem.ETKDG()) != 0:
        print(f"Embedding failed for SMILES at line {idx + 1}: {smiles}")
        continue
    AllChem.UFFOptimizeMolecule(mol)
    
    sdf_path = os.path.join(output_dir, f"ligand_{idx}.sdf")
    writer = Chem.SDWriter(sdf_path)
    writer.write(mol)
    writer.close()

print(f"Finished writing {len(smiles_list)} SDF files to '{output_dir}'")

In [ ]:
#run diffdock
import os
import requests
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

# ---- CONFIG ----
input_dir = "sdf_output"
output_dir = "results_output"
receptor_path = "receptor_clean.pdb"

url = "https://health.api.nvidia.com/v1/biology/mit/diffdock"
header_auth = "Bearer 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"

# ---- ASSET UPLOAD FUNCTION ----
def _upload_asset(input_data):
    assets_url = "https://api.nvcf.nvidia.com/v2/nvcf/assets"
    headers = {
        "Authorization": header_auth,
        "Content-Type": "application/json",
        "accept": "application/json",
    }
    s3_headers = {
        "x-amz-meta-nvcf-asset-description": "diffdock-file",
        "content-type": "text/plain",
    }
    payload = {
        "contentType": "text/plain",
        "description": "diffdock-file"
    }

    for attempt in range(5):  # retry up to 5 times
        try:
            response = requests.post(assets_url, headers=headers, json=payload, timeout=30)
            response.raise_for_status()
            asset_url = response.json()["uploadUrl"]
            asset_id = response.json()["assetId"]

            response = requests.put(asset_url, data=input_data, headers=s3_headers, timeout=300)
            response.raise_for_status()

            return asset_id
        except requests.exceptions.HTTPError as e:
            if response.status_code == 429:
                wait = 2 ** attempt + random.uniform(0, 1)
                print(f"[WARN] Rate limited. Retrying after {wait:.2f}s...")
                time.sleep(wait)
            else:
                raise
    raise RuntimeError("Failed to upload asset after multiple attempts")

# ---- UPLOAD PROTEIN ONCE ----
with open(receptor_path, "rb") as f:
    protein_id = _upload_asset(f.read())
print(f"Protein uploaded: {protein_id}")

# ---- PROCESS ONE LIGAND ----
def process_ligand(idx, sdf_file):
    try:
        ligand_path = os.path.join(input_dir, sdf_file)
        out_folder = os.path.join(output_dir, f"ligand_{idx}")
        os.makedirs(out_folder, exist_ok=True)

        with open(ligand_path, "rb") as f:
            ligand_id = _upload_asset(f.read())

        print(f"Ligand {sdf_file} uploaded: {ligand_id}")

        headers = {
            "Content-Type": "application/json",
            "NVCF-INPUT-ASSET-REFERENCES": f"{protein_id},{ligand_id}",
            "Authorization": header_auth
        }

        payload = {
            "ligand": ligand_id,
            "ligand_file_type": "sdf",
            "protein": protein_id,
            "num_poses": 20,
            "time_divisions": 20,
            "steps": 18,
            "save_trajectory": True,
            "is_staged": True
        }

        for attempt in range(5):  # Retry logic for rate-limited inference
            response = requests.post(url, headers=headers, json=payload)
            if response.status_code != 429:
                break
            wait = 2 ** attempt + random.uniform(0, 1)
            print(f"[WARN] Inference rate-limited. Retrying after {wait:.2f}s...")
            time.sleep(wait)

        with open(os.path.join(out_folder, "response_status.txt"), "w") as f:
            f.write(str(response))

        with open(os.path.join(out_folder, "request_url.txt"), "w") as f:
            f.write(url)

        with open(os.path.join(out_folder, "response_text.txt"), "w") as f:
            f.write(response.text)

        print(f"Completed ligand_{idx}: {response.status_code}")
    except Exception as e:
        print(f"[ERROR] Ligand {idx} failed: {e}")

# ---- MULTITHREADING EXECUTION ----
os.makedirs(output_dir, exist_ok=True)
sdf_files = [f for f in os.listdir(input_dir) if f.endswith(".sdf")]

# Reduce concurrency to avoid 429 errors
max_workers = min(3, len(sdf_files))  # Try 2–3 threads instead of 8

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_ligand, idx, sdf_file) for idx, sdf_file in enumerate(sdf_files)]
    for future in as_completed(futures):
        pass  # Ensures we wait for all tasks

In [ ]:
#reformatting the output. select the top 100 based on confidence score. extract their smiles string.
import json
import os
from rdkit import Chem
from rdkit.Chem import AllChem

base_path = r"results_output"
ligand_confidences = []

for i in range(40000):
    ligand_folder = os.path.join(base_path, f"ligand_{i}")
    input_file = os.path.join(ligand_folder, "response_text.txt")
    output_folder = os.path.join(ligand_folder, "diffdock_actual_outcome")

    if not os.path.exists(input_file):
        continue

    os.makedirs(output_folder, exist_ok=True)

    with open(input_file, "r") as f:
        data = json.load(f)

    # Write PDB and SDF
    for j, pose in enumerate(data.get("trajectory", []), start=1):
        with open(os.path.join(output_folder, f"pose_{j}.pdb"), "w") as pdb_file:
            pdb_file.write(pose)

    for j, sdf in enumerate(data.get("ligand_positions", []), start=1):
        with open(os.path.join(output_folder, f"ligand_pose_{j}.sdf"), "w") as sdf_file:
            sdf_file.write(sdf)

    # Write confidence scores
    confidences = data.get("position_confidence", [])
    with open(os.path.join(output_folder, "pose_confidences.txt"), "w") as out_file:
        out_file.write("Rank \t Pose Confidence\n\n")
        for j, conf in enumerate(confidences, start=1):
            out_file.write(f"{j} \t {conf}\n")

    valid_confidences = [c for c in confidences if c is not None]
    if valid_confidences:
        highest = max(valid_confidences)
        ligand_confidences.append((i, highest))

# Sort and select top 100 ligands
ligand_confidences.sort(key=lambda x: x[1], reverse=True)
top_100_ligands = ligand_confidences[:100]

# Save ligand numbers and confidence scores
conf_out_path = os.path.join(base_path, "top_100_ligand_confidences.txt")
with open(conf_out_path, "w") as out:
    out.write("Ligand Number\tConfidence Score\n")
    for num, score in top_100_ligands:
        out.write(f"{num}\t{score:.4f}\n")

# Convert PDB to SMILES and save
smiles_out_path = os.path.join(base_path, "Lipinski_before.txt")
with open(smiles_out_path, "w") as out:
    out.write("Ligand Number\tSMILES\n")
    for num, _ in top_100_ligands:
        pdb_path = os.path.join(base_path, f"ligand_{num}", "diffdock_actual_outcome", "ligand_pose_1.pdb")
        if os.path.exists(pdb_path):
            mol = Chem.MolFromPDBFile(pdb_path, removeHs=False)
            if mol is not None:
                try:
                    AllChem.Compute2DCoords(mol)
                    smiles = Chem.MolToSmiles(mol)
                    out.write(f"{num}\t{smiles}\n")
                except:
                    out.write(f"{num}\tERROR: Failed to generate SMILES\n")
            else:
                out.write(f"{num}\tERROR: Invalid molecule\n")
        else:
            out.write(f"{num}\tERROR: PDB file missing\n")

print("Output written to:")
print(conf_out_path)
print(smiles_out_path)



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\zhaol\\Downloads\\drug_discovery_pipeline\\Docking Pipeline\\results_output\\top_100_ligand_confidences.txt'

#
input: Lipinski_before.txt
output:Lipinski_after.txt

In [3]:
from rdkit import Chem
from rdkit.Chem import Descriptors

def lipinski_violations(mol):
    """Return the count of Lipinski rule violations."""
    mw   = Descriptors.MolWt(mol)
    logp = Descriptors.MolLogP(mol)
    hbd  = Descriptors.NumHDonors(mol)
    hba  = Descriptors.NumHAcceptors(mol)

    violations = 0
    if mw   >= 500: violations += 1
    if logp >= 5:   violations += 1
    if hbd  >= 5:   violations += 1
    if hba  >= 10:  violations += 1

    return violations

def filter_lipinski(input_path='Lipinski_before.txt', output_path='Lipinski_after.txt'):
    passed = []
    with open(input_path, 'r') as infile:
        for line in infile:
            smi = line.strip()
            if not smi:
                continue

            mol = Chem.MolFromSmiles(smi)
            if mol is None:
                continue

            if lipinski_violations(mol) <= 1:
                passed.append(smi)

    with open(output_path, 'w') as outfile:
        for smi in passed:
            outfile.write(smi + '\n')

    print(f"Saved {len(passed)} molecules that passed Lipinski’s rule to `{output_path}`")

if __name__ == '__main__':
    filter_lipinski()

Saved 40924 molecules that passed Lipinski’s rule to `Lipinski_after.txt`


check binding pocket via autodock